<a href="https://colab.research.google.com/github/HarshGujarathi7/Data-Science-in-Finance--Independent-Study/blob/main/Fraud_Detection_Data_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Taking 4 datsets from paper https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10535547/#sec3dot2-sensors-23-07788

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
